# Сравнение выручки на пользователя с помощью распределения Парето

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

from collections import namedtuple

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

Распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] 

$$
P(x; x_m, c) = 
\begin{cases}
\dfrac{c x_m^c}{x^{c + 1}} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for b, loc, scale in [(1, 0, 1), (2, 0, 1), (3, 0, 4), (4, 1, 1)]:
    fig.add_trace(go.Scatter(x=x, y=stats.pareto.pdf(x, b=b, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f'b={b}, loc={loc}, x_m={scale}'))
fig.update_layout(title='Pareto',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}.
$$

Сопряженное априорное распределение - Гамма-распределение
https://en.wikipedia.org/wiki/Conjugate_prior


$$
P(model) = Gamma(x; a, b) = \frac{b^a}{\Gamma(a)} x^{a-1} e^{-b x}, \quad x>0, \quad a,b>0 .
$$

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \alpha) = 
\begin{cases}
\dfrac{\alpha}{x_m} \left(\dfrac{x}{x_m}\right)^{-(\alpha+1)} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

$$
P(model) = Gamma(\alpha; a, b) = \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}, \quad \alpha>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{\alpha}{x_m}\left(\frac{x}{x_m}\right)^{-(\alpha + 1)} \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto 
\alpha e^{-(\alpha+1) \ln(x/x_m)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto
\alpha^{a} e^{-\alpha (b + \ln(x/x_m))}
\\
& \sim
Gamma(\alpha; a+1, b + \ln(x/x_m))
\end{split}
$$

Для N точек
$$
P(model | data) = Gamma(\alpha; a+N, b + \sum_i^N\ln(x_i/x_m))
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for a,b in [(1,1), (2,1), (3,1), (9, 3), (45, 15), (90, 30)]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
fig.update_layout(title='Gamma Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

# Оценка формы распределения Парето

In [ ]:
def posterior_params(alpha, beta, x_m, data):
    alpha_post = alpha + len(data)
    beta_post = beta + np.sum([np.log(x/x_m) for x in data])
    return alpha_post, beta_post

def posterior_dist(alpha, beta, x_m, data):
    alpha_post = alpha + len(data)
    beta_post = beta + np.sum([np.log(x/x_m) for x in data])
    dist = stats.gamma(a=alpha_post, scale=1/beta_post)
    return dist

b = 2.8
scale = 1
nsample = 1000

exact_dist = stats.pareto(b=b)

data = exact_dist.rvs(nsample)

alpha_prior = 2
beta_prior = 1

postdist = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=data)
#postdist

x = np.linspace(0, 10, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist.pdf(x), mode='lines', name=f'Posterior'))
fig.update_layout(title='Gamma Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

Апостериорное распределение

Среднее

$$
E[x] = 
\begin{cases}
\dfrac{\alpha x_m}{\alpha - 1} & \alpha > 1
\\
\infty & \alpha \le 1
\end{cases}
$$

Медиана

$$
m = x_m 2^{1/\alpha}
$$

In [ ]:
alpha_nsamp = 50000
samp = postdist.rvs(alpha_nsamp)
samp_mean = samp * scale / (samp-1)
samp_median = scale * 2**(1/samp)

fig = go.Figure()
fig.add_trace(go.Histogram(x=samp_mean, histnorm='probability density', name='Mean Hist', nbinsx=100))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.show()

fig = go.Figure()
fig.add_trace(go.Histogram(x=samp_median, histnorm='probability density', name='Median Hist', nbinsx=100))
fig.add_vline(exact_dist.median(), name='Original Distribution Median')
fig.show()

# Сравнение распределений

In [ ]:
# params = {
#     'A': {'b': 2.8}
#     'B': {'b': 2.9}
# }

b = 1.3
scale = 1

exact_dist_a = stats.pareto(b=b)
exact_dist_b = stats.pareto(b=b*1.05)

nsample = 1000

samp_a = exact_dist_a.rvs(nsample)
samp_b = exact_dist_b.rvs(nsample)

alpha_prior = 2
beta_prior = 1

postdist_a = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=samp_a)
postdist_b = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=samp_b)
#postdist

x = np.linspace(0, 5, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_a.pdf(x), mode='lines', name=f'A'))
fig.add_trace(go.Scatter(x=x, y=postdist_b.pdf(x), mode='lines', name=f'B'))
fig.update_layout(title='Posterior Distribution',
                  xaxis_title='$alpha$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


post_nsamp = 50000
post_samp_a = postdist_a.rvs(alpha_nsamp)
post_samp_b = postdist_b.rvs(alpha_nsamp)
p_alpha_b_gt_alpha_a = np.sum(post_samp_b > post_samp_a) / post_nsamp
print(f'P(alpha_B > alpha_A) = {p_alpha_b_gt_alpha_a :.2f}')

# Приложение

Ломакс

$$
p(x) = {\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)}, \qquad x \geq 0
$$

$$
p(x) = {{\alpha\lambda^\alpha} \over {(x + \lambda)^{\alpha+1}}}
$$

Для распределения на пользователей: обезразмерить на $x_m$.  
Потом Ломакс с $\lambda=1$.  
Проще сразу безразмерный вариант распределения.

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \alpha) = {\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)}
= \frac{\alpha}{\lambda}\frac{1}{\left[{1 + (x/\lambda)}\right]^{(\alpha+1)}}
$$

$$
P(model) = Gamma(\alpha; a, b) = \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}, \quad \alpha>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
{\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)} \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto 
\alpha e^{-(\alpha+1) \ln(1 + x/\lambda)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto
\alpha^{a} e^{-\alpha (b + \ln(1 + x/\lambda))}
\\
& \sim
Gamma(\alpha; a+1, b + \ln(1 + x/\lambda))
\end{split}
$$

Для N точек
$$
P(model | data) = Gamma(\alpha; a+N, b + \sum_i^N\ln(1 + x_i/\lambda))
$$